In [4]:
import json

In [5]:
!pip install sacrebleu

In [6]:
import torch
torch.cuda.is_available()

True

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "Mhassanen/nllb-200-600M-En-Ar"
tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang="eng_Latn", tgt_lang="arz_Arab")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)



In [8]:
import pandas as pd
import glob


pip install pandas openpyxl



In [ ]:

# Define input folder (where the .xlsx files are)
data_file_folder = r"C:\Bachelor\Week 5\Sample_Data\Novels" # Update with your actual path

# Define output folder and file
output_folder = "C:\Bachelor\Week 5\Merged_Data"
output_file = os.path.join(output_folder, "merged_data.xlsx")

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# List to store dataframes
df_list = []

# Read all .xlsx files and force data into two columns (A & B)
for file in os.listdir(data_file_folder):
    if file.endswith('.xlsx'):
        print(f'Loading file {file}...')
        file_path = os.path.join(data_file_folder, file)
        try:
            # Read the Excel file without headers
            df = pd.read_excel(file_path, sheet_name=0, header=None, engine="openpyxl")

            # Ensure we only take the first two columns
            df = df.iloc[:, :2]

            # Rename columns to keep them uniform
            df.columns = ["ColumnA", "ColumnB"]

            # Append to list
            df_list.append(df)
        except Exception as e:
            print(f"Error loading {file}: {e}")

# Concatenate all dataframes under A & B columns
if df_list:
    merged_df = pd.concat(df_list, ignore_index=True)

    # Save to new Excel file
    merged_df.to_excel(output_file, index=False)
    print(f"✅ Merged file saved at: {output_file}")
else:
    print("❌ No valid Excel files found.")


NameError: name 'os' is not defined

In [10]:
data_file = r"C:\Bachelor\Week 5\Sample_Data\Novels\the-little-prince.xlsx"  # Update with your actual path
df = pd.read_excel(data_file)

In [11]:
print(df.head())

                                       الأمير الصغير  \
0                            أنطوان دي سانت إكزوبيري   
1                                        الفصل الأول   
2  في مره، لما كان عندي ست سنين، شفت صوره روعه، ف...   
3      وكانت عباره عن تعبان من نوع البوا بيبلع فهد.    
4                                ودي نسخة من الرسمه.   

                                  The little prince   
0                            Antoine De Saint-Exuper  
1                                          Chapter 1  
2  Once when I was six years old I saw a magnific...  
3  It was a picture of a boa constrictor in the a...  
4                     Here is a copy of the drawing.  


In [12]:
df.columns = ["English", "Arabic"]

Evaluation Installation


In [13]:
from IPython import get_ipython
get_ipython().system('python -c "import evaluate; print(evaluate.load(\'exact_match\').compute(references=[\'hello\'], predictions=[\'hello\']))"')

{'exact_match': 1.0}


In [24]:


df = pd.read_excel(data_file)  # Adjust sheet_name if needed

# Assuming the first column contains English sentences and the second column has Arabic translations
english_sentences = df.iloc[:, 1].tolist()  # Extract English sentences
arabic_sentences = df.iloc[:, 0].tolist()  # Extract Arabic translations

print("Sample Data:")
print("English:", english_sentences[:3])
print("Arabic:", arabic_sentences[:3])


Sample Data:
English: ['Antoine De Saint-Exuper', 'Chapter 1', 'Once when I was six years old I saw a magnificent picture in a book, called True Stories from Nature, about the primeval forest.']
Arabic: ['أنطوان دي سانت إكزوبيري', 'الفصل الأول', 'في مره، لما كان عندي ست سنين، شفت صوره روعه، في كتاب عن الغابات إللي ماحدش زارها قبل كدا كان اسمه قصص حقيقيه. ']


Torch Installation


In [1]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.version.cuda)  # Should show 12.1
print(torch.cuda.get_device_name(0))  # Should show your GPU name


True
12.1
NVIDIA GeForce RTX 3060 Laptop GPU


In [21]:
def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    translated_tokens = model.generate(**inputs)
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    return translated_text

GPU 


In [ ]:
model.to("cuda")  # Move model to GPU

In [ ]:
def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True).to("cuda")  # Move input to GPU
    translated_tokens = model.generate(**inputs)
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    return translated_text

In [16]:
predictions = [translate(text) for text in english_sentences]

With execution time

In [ ]:
from tqdm import tqdm

predictions = [translate(text) for text in tqdm(english_sentences, desc="Translating")]


In [18]:
import evaluate

In [ ]:
bleu = evaluate.load("bleu")

# Format references correctly: Each reference sentence must be inside a list
references = [[text] for text in arabic_sentences]




In [22]:
import evaluate

bleu = evaluate.load("bleu")

# Ensure predictions are strings and references are lists of lists
predictions = [str(pred).strip() for pred in predictions]  
formatted_references = [[str(ref).strip()] for ref in references]  

# Compute BLEU score
results = bleu.compute(predictions=predictions, references=formatted_references)

print("BLEU Score:", results["bleu"])


BLEU Score: 0.06861565822114636


Testing Accuracy on Lyrics Data


In [26]:
data_file2 = r"C:\Bachelor\Week 5\Sample_Data\Songs\Song-Lyrics.xlsx"  # Update with your actual path
df2 = pd.read_excel(data_file2)

In [29]:


df2 = pd.read_excel(data_file2)  # Adjust sheet_name if needed

# Assuming the first column contains English sentences and the second column has Arabic translations
english_sentences2 = df2.iloc[:, 5].tolist()  # Extract English sentences
arabic_sentences2 = df2.iloc[:, 4].tolist()  # Extract Arabic translations

print("Sample Data:")
print("English:", english_sentences2[:3])
print("Arabic:", arabic_sentences2[:3])


Sample Data:
English: ['The heart of the city teems with sorrow.', 'The eyes of the people are lost, they hide in the mist.', 'Everyone dances to the beat of their own drums.']
Arabic: ['المدينة قلبها زحمة بالهموم', 'والعيون تايهه بتداري في الغيوم', 'مين يهمه إلا همه؟']


In [30]:
model.to("cuda")

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100SdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
   

In [31]:
def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True).to("cuda")  # Move input to GPU
    translated_tokens = model.generate(**inputs)
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    return translated_text

In [32]:
from tqdm import tqdm

predictions = [translate(text) for text in tqdm(english_sentences, desc="Translating")]


Translating: 100%|██████████| 1481/1481 [08:40<00:00,  2.85it/s]


In [33]:
import evaluate
bleu = evaluate.load("bleu")

# Format references correctly: Each reference sentence must be inside a list
references = [[text] for text in arabic_sentences]




In [34]:
import evaluate

bleu = evaluate.load("bleu")

# Ensure predictions are strings and references are lists of lists
predictions = [str(pred).strip() for pred in predictions]  
formatted_references = [[str(ref).strip()] for ref in references]  

# Compute BLEU score
results = bleu.compute(predictions=predictions, references=formatted_references)

print("BLEU Score:", results["bleu"])


BLEU Score: 0.06861565822114636
